#### Data 612 - Project 1<br>Christina Valore, Juliann McEachern, Rajwant Mishra<br>Due: June 11, 2019

# Global Baseline Predictors and RMSE

## Overview

In this first assignment, we'll attempt to predict ratings with very little information.  We'll first look at just raw averages across all (training dataset) users.  We'll then account for "bias" by normalizing across users and across items.   You'll be working with ratings in a user-item matrix, where each rating may be (1) assigned to a training dataset, (2) assigned to a test dataset, or (3) missing. 

Please code as much of your work as possible in R or Python.  You may use standard functions (e.g. from base R and the tidyverse).  Your project should be delivered in an R Markdown or a Jupyter notebook, then the notebook should be saved into a GitHub repository.  You should include a link to your GitHub repository in your assignment submission link. 

## Recommender System Data Selection

We built a recommender system that suggests movies based on IMBD rating scores. The following movies were selected and scraped from IMBD using R: 

![Example](Data/Movie_Titles.JPG)

The Rmd file and output datasets are stored in the `Data` file within this repository. The dataset includes missing ratings across all movies as not all users rated all selected titles. 

## Data Preparation 

In [11]:
import pandas as pd
import numpy as np

#load data from csv in github repository into pandas dataframe
data = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-1/Data/User%20_Mov%20.csv')

#preview imported data
data.head()

ID  Movie_ID         User_Name  Rating  \
0   1         1  TheTopDawgCritic       8   
1   2         1  celestinoavilajr       8   
2   3         1      dave-mcclain       8   
3   4         1        neener3707       7   
4   5         1           krice23       8   

                                           Title  \
0                          Better than expected!   
1                                  Entertaining!   
2                    an expectedly good thriller   
3  Done Before, But Not Too Bad, A Little Extra    
4                                          Sold!   

                                             Content  \
0  For a B-grade film, this was surprisingly bett...   
1  Good movie! Great plot! Edge of your seat twis...   
2                                   Bad Samaritan" R   
3  Over all not too shabby, definitely better tha...   
4  Original script. Believable cast. They make it...   

                                          Unnamed: 6  \
0                                                NaN   
1                                                NaN   
2   151 is a thriller directed by actor-turned-pr...   
3                     Im not sure what I want to say   
4                                                NaN   

                                          Unnamed: 7  \
0                                                NaN   
1                                                NaN   
2   written by Brandon Boyce "Wicker Park" and st...   
3                   because some of it was effective   
4                                                NaN   

                                          Unnamed: 8  \
0                                                NaN   
1                                                NaN   
2   but this film is primarily an intricately plo...   
3                 and some of it wasnt. First of all   
4                                                NaN   

                                          Unnamed: 9     ...     Unnamed: 15  \
0                                                NaN     ...             NaN   
1                                                NaN     ...             NaN   
2   a young man whose step-father brought him and...     ...             NaN   
3   I like how they expanded on the typical story...     ...             NaN   
4                                                NaN     ...             NaN   

  Unnamed: 16 Unnamed: 17 Unnamed: 18 Unnamed: 19 Unnamed: 20 Unnamed: 21  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN         NaN   

  Unnamed: 22 Unnamed: 23 Unnamed: 24  
0         NaN         NaN         NaN  
1         NaN         NaN         NaN  
2         NaN         NaN         NaN  
3         NaN         NaN         NaN  
4         NaN         NaN         NaN  

[5 rows x 25 columns]

In [5]:
#remove unnecessary columns, including blank columns
data=data.drop(data.columns.to_series()[4:25], axis=1)

#create small subset of the data for manual calculations, 5 rows to be exact
subst=data.iloc[:5,:]

#split the dataframe into training and test dataframes from rows 1-250 and from 250-687
train=data.iloc[:250, :]
test=data.iloc[250:, :]

In [9]:
#preview train dataframe
train.head()

ID  Movie_ID         User_Name  Rating
0   1         1  TheTopDawgCritic       8
1   2         1  celestinoavilajr       8
2   3         1      dave-mcclain       8
3   4         1        neener3707       7
4   5         1           krice23       8

In [10]:
#preview test dataframe
test.head()

ID  Movie_ID         User_Name  Rating
250  252        11       gbill-74877       8
251  253        11           atlasmb       8
252  254        11  steve-552-934420       1
253  255        11        dfranzen70       8
254  256        11          poetschn       5

In [93]:
#view the subsetted data
subst

ID  Movie_ID         User_Name  Rating
0   1         1  TheTopDawgCritic       8
1   2         1  celestinoavilajr       8
2   3         1      dave-mcclain       8
3   4         1        neener3707       7
4   5         1           krice23       8

In [94]:
#view the end of the train data
train.tail()

ID  Movie_ID       User_Name  Rating
245  247        11           zofos       5
246  248        11         radivel       9
247  249        11      Spikeopath       9
248  250        11  blackmambamark       8
249  251        11  kellyfacebook1       5

In [95]:
#view the end of the test data
test.tail()

ID  Movie_ID          User_Name  Rating
682  684         6  bob-the-movie-man       4
683  685         6         lorriejinx       1
684  686         6           plargreg       1
685  687         6         luke-solly       1
686  688         6           viraltux       4

## RSME 
*  Using your training data, calculate the raw average (mean) rating for every user-item combination. 
*  Calculate the RMSE for raw average for both your training data and your test data.
*  Using your training data, calculate the bias for each user and each item.
*  From the raw average, and the appropriate user and item biases, calculate the baseline predictors for every user-item combination. 
*  Calculate the RMSE for the baseline predictors for both your training data and your test data.

## Summary 
*  Summarize your results

In [96]:
train.columns


Index(['ID', 'Movie_ID', 'User_Name', 'Rating'], dtype='object')

In [145]:
train = train[['Movie_ID', 'User_Name', 'Rating']]

In [97]:
# train['User_Name'].unique
# train['User_Name'].nunique

* Normalize the score .by create User Item rating matrix 

In [146]:
df_matrix_item = pd.pivot_table(train, values='Rating', index='User_Name', columns='Movie_ID')
df_matrix_item.head()

Movie_ID           1   2     3   4    5   6     7   8   9     10   11
User_Name                                                            
3xHCCH            NaN NaN   NaN NaN  NaN NaN  10.0 NaN NaN   NaN  NaN
AirBourne_Bds     NaN NaN   NaN NaN  NaN NaN   NaN NaN NaN  10.0  NaN
Ajk2386           NaN NaN   NaN NaN  6.0 NaN   NaN NaN NaN   NaN  NaN
AnonymousMouseMan NaN NaN  10.0 NaN  NaN NaN   NaN NaN NaN   NaN  NaN
Apemangalore      NaN NaN   NaN NaN  NaN NaN   NaN NaN NaN   NaN  9.0

In [147]:
# df_matrix_item.to_excel('test.xlsx') 
df_matrix_item.mean().head()



Movie_ID
1    7.708333
2    8.280000
3    7.208333
4    5.200000
5    3.560000
dtype: float64

In [149]:
train.head(10)

Movie_ID            User_Name  Rating
0         1     TheTopDawgCritic       8
1         1     celestinoavilajr       8
2         1         dave-mcclain       8
3         1           neener3707       7
4         1              krice23       8
5         1       patrickc-76383       7
6         1       george.schmidt       7
7         1  godsgirlfrombluford       9
8         1     claudio_carvalho       8
9         1    Stars-and-popcorn       6

In [310]:
Mov_Info = train.groupby(['Movie_ID']).mean().rename(columns={'Rating':'Mov_Info_Rating'})

# Calculating Bias for Mov.
Mov_Info['Mov_Bias'] = (trainb[['Movie_ID','Rating']].groupby(['Movie_ID']).mean()-Train_Raw_Avg).rename(
    columns={'Rating':'Mov_Bias'})



Mov_Info.head()


Mov_Info_Rating  Mov_Bias
Movie_ID                           
1                7.708333  0.816333
2                8.280000  1.388000
3                7.208333  0.316333
4                5.200000 -1.692000
5                3.560000 -3.332000

In [311]:
Usr_Rate_Info = train[['User_Name','Rating']].groupby(['User_Name']).mean().rename(columns={'Rating':'Usr_Mean_Rating'})

Usr_Rate_Info.head()

# Count the number of rating given by User 
Usr_Rate_Info = train[['User_Name','Rating']].groupby(['User_Name']).count().rename(columns={'Rating':'Usr_Rate_Count'})

# Same this in One Line 
# Calculating the Mean of user's rating and count the numbner of movies a user has rated.

Usr_Rate_Info = train[['User_Name','Rating']].groupby(['User_Name']).agg(
    {'Rating':{ 'mean',  'min'}, 'User_Name':{'count'}}).rename(columns={'min':'Usr_Min_Rating',
                                             'count':'Usr_Rate_Count',
                                             'mean' : 'Usr_Mean_Rating'})


Usr_Rate_Info.head()


Rating                     User_Name
                  Usr_Mean_Rating Usr_Min_Rating Usr_Rate_Count
User_Name                                                      
3xHCCH                       10.0             10              1
AirBourne_Bds                10.0             10              1
Ajk2386                       6.0              6              1
AnonymousMouseMan            10.0             10              1
Apemangalore                  9.0              9              1

In [312]:
# Using drop level to get the dataframe 
Usr_Rate_Info.columns = Usr_Rate_Info.columns.droplevel(0)
# Calculating Bias for User 
Usr_Rate_Info['Usr_Bias'] = (trainb[['User_Name','Rating']].groupby(['User_Name']).mean()-Train_Raw_Avg).rename(columns={'Rating':'Usr_Bias'})

Usr_Rate_Info.head()

Usr_Mean_Rating  Usr_Min_Rating  Usr_Rate_Count  Usr_Bias
User_Name                                                                   
3xHCCH                        10.0              10               1     3.108
AirBourne_Bds                 10.0              10               1     3.108
Ajk2386                        6.0               6               1    -0.892
AnonymousMouseMan             10.0              10               1     3.108
Apemangalore                   9.0               9               1     2.108

In [319]:
# Merge the User Rating Mean and count info from Usr_
trainb = pd.merge(train,Usr_Rate_Info,on='User_Name')
trainb.head(16)

Movie_ID            User_Name  Rating  Usr_Mean_Rating  Usr_Min_Rating  \
0          1     TheTopDawgCritic       8              8.0               8   
1          3     TheTopDawgCritic       8              8.0               8   
2          4     TheTopDawgCritic       8              8.0               8   
3          1     celestinoavilajr       8              8.0               8   
4          1         dave-mcclain       8              8.0               8   
5          1           neener3707       7              5.0               3   
6          4           neener3707       3              5.0               3   
7          1              krice23       8              8.0               8   
8          1       patrickc-76383       7              7.0               7   
9          1       george.schmidt       7              7.0               7   
10         1  godsgirlfrombluford       9              9.0               9   
11         1     claudio_carvalho       8              5.0               1   
12         3     claudio_carvalho       6              5.0               1   
13         4     claudio_carvalho       3              5.0               1   
14         7     claudio_carvalho       7              5.0               1   
15        10     claudio_carvalho       1              5.0               1   

    Usr_Rate_Count  Usr_Bias  
0                3     1.108  
1                3     1.108  
2                3     1.108  
3                1     1.108  
4                1     1.108  
5                2    -1.892  
6                2    -1.892  
7                1     1.108  
8                1     0.108  
9                1     0.108  
10               1     2.108  
11               5    -1.892  
12               5    -1.892  
13               5    -1.892  
14               5    -1.892  
15               5    -1.892

In [320]:
trainb = pd.merge(trainb,Mov_Info,on='Movie_ID')


In [321]:
trainb.head()

Movie_ID         User_Name  Rating  Usr_Mean_Rating  Usr_Min_Rating  \
0         1  TheTopDawgCritic       8              8.0               8   
1         1  celestinoavilajr       8              8.0               8   
2         1      dave-mcclain       8              8.0               8   
3         1        neener3707       7              5.0               3   
4         1           krice23       8              8.0               8   

   Usr_Rate_Count  Usr_Bias  Mov_Info_Rating  Mov_Bias  
0               3     1.108         7.708333  0.816333  
1               1     1.108         7.708333  0.816333  
2               1     1.108         7.708333  0.816333  
3               2    -1.892         7.708333  0.816333  
4               1     1.108         7.708333  0.816333

In [322]:
# # user -Item utility Matrix
df_User_Mov = pd.pivot_table(trainb, values='Rating', index='User_Name', columns='Movie_ID',fill_value="-")
df_User_Mov.head(50)
# trainb = trainb.reindex(df_pivot['Movie_ID'].sort_values(by=['Usr_Rate_Count'],ascending=False).index)

# #Soritng Output by Rating Count in Descending order
Check_Trainb = trainb.sort_values(by=['Usr_Rate_Count'],ascending=False)
trainb[(trainb.Usr_Rate_Count > 2)]
pd.pivot_table(trainb[(trainb.Usr_Rate_Count > 1)], values='Rating', index='User_Name', columns='Movie_ID',fill_value="-")


Movie_ID         1  2  3  4  7  8  9  10 11
User_Name                                  
Lady_Targaryen    -  -  -  -  -  9  7  -  -
Platypuschow      7  9  6  -  -  -  -  -  -
TheTopDawgCritic  8  -  8  8  -  -  -  -  -
claudio_carvalho  8  -  6  3  7  -  -  1  -
neener3707        7  -  -  3  -  -  -  -  -
the-movie-guy     -  -  -  -  8  -  -  -  7

In [323]:
pd.pivot_table(trainb[(trainb.Usr_Rate_Count > 1)], values='Usr_Mean_Rating', index='User_Name', columns='Movie_ID',fill_value="-")


Movie_ID               1        2        3  4    7  8  9  10   11
User_Name                                                        
Lady_Targaryen          -        -        -  -    -  8  8  -    -
Platypuschow      7.33333  7.33333  7.33333  -    -  -  -  -    -
TheTopDawgCritic        8        -        8  8    -  -  -  -    -
claudio_carvalho        5        -        5  5    5  -  -  5    -
neener3707              5        -        -  5    -  -  -  -    -
the-movie-guy           -        -        -  -  7.5  -  -  -  7.5

# -------------------------------------------

# -------------------------------------------

In [279]:
# calcualting https://www.youtube.com/watch?v=prVRuPezW3Q&list=PLuKhJYywjDe96T2L0-zXFU5Up2jqXlWI9&index=12
# Using Raw Average 
Train_Raw_Avg = train['Rating'].mean()
Train_Raw_Avg

6.892

In [284]:
# Calcualte the New rating based on Users Mean Rating 
# Lets assumae Usrs Mean is the rating of the movie and Calcuate the RMSE of the Train  data.
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

print ('RMSE for the Train data using User\'s Mean Rating: ', rmse(trainb.Rating,trainb.Usr_Mean_Rating))


RMSE for the Train data using User's Mean Rating:  0.443471156521669


In [285]:
trainb['Raw_Avg']= Train_Raw_Avg
print ('RMSE for the Train data using Raw Avg of the Movie: ', rmse(trainb.Rating,trainb.Raw_Avg))

RMSE for the Train data using Raw Avg of the Movie:  3.103600489753794


# Base line predictor 
Raw Avg + Bias User + Bias of Movie


In [325]:
#--------------------------------------------------
# Users Avg 
# Single User's Avg- Raw Avg 
#--------------------------------------------------
# Movie Avg
# Single Movie's avg - Rav Avg
#--------------------------------------------------

trainb.head()

Movie_ID         User_Name  Rating  Usr_Mean_Rating  Usr_Min_Rating  \
0         1  TheTopDawgCritic       8              8.0               8   
1         1  celestinoavilajr       8              8.0               8   
2         1      dave-mcclain       8              8.0               8   
3         1        neener3707       7              5.0               3   
4         1           krice23       8              8.0               8   

   Usr_Rate_Count  Usr_Bias  Mov_Info_Rating  Mov_Bias  
0               3     1.108         7.708333  0.816333  
1               1     1.108         7.708333  0.816333  
2               1     1.108         7.708333  0.816333  
3               2    -1.892         7.708333  0.816333  
4               1     1.108         7.708333  0.816333

In [348]:
# TO build Baseline We will take Raw average + User's Bias Rating + Mov Bias Rating
trainb['Baseline_Pred'] =  (Train_Raw_Avg +  trainb.Usr_Bias + trainb.Mov_Bias)


# Very Imp step this reduced RMSE .20 by just removing the error and reseting any rating more than 10 to 10.
trainb['Baseline_Pred'] = np.where(trainb['Baseline_Pred'] >=10, 10,trainb['Baseline_Pred'] )

trainb.head()

Movie_ID         User_Name  Rating  Usr_Mean_Rating  Usr_Min_Rating  \
0         1  TheTopDawgCritic       8              8.0               8   
1         1  celestinoavilajr       8              8.0               8   
2         1      dave-mcclain       8              8.0               8   
3         1        neener3707       7              5.0               3   
4         1           krice23       8              8.0               8   

   Usr_Rate_Count  Usr_Bias  Mov_Info_Rating  Mov_Bias  Baseline_Pred  
0               3     1.108         7.708333  0.816333       8.816333  
1               1     1.108         7.708333  0.816333       8.816333  
2               1     1.108         7.708333  0.816333       8.816333  
3               2    -1.892         7.708333  0.816333       5.816333  
4               1     1.108         7.708333  0.816333       8.816333

In [350]:
print ('RMSE for the Train data using Baseline Predictor: ', rmse(trainb.Rating,trainb.Baseline_Pred))

RMSE for the Train data using Baseline Predictor:  1.5480927938537123


In [351]:
# user Move Matrix 
pd.pivot_table(train, values='Rating', index='User_Name', columns='Movie_ID',fill_value="-").head()

Movie_ID          1  2   3  4  5  6   7  8  9   10 11
User_Name                                            
3xHCCH             -  -   -  -  -  -  10  -  -   -  -
AirBourne_Bds      -  -   -  -  -  -   -  -  -  10  -
Ajk2386            -  -   -  -  6  -   -  -  -   -  -
AnonymousMouseMan  -  -  10  -  -  -   -  -  -   -  -
Apemangalore       -  -   -  -  -  -   -  -  -   -  9

In [352]:
pd.pivot_table(trainb, values='Baseline_Pred', index='User_Name', columns='Movie_ID',fill_value="-").head()

Movie_ID          1  2   3  4      5  6   7  8  9      10  11
User_Name                                                    
3xHCCH             -  -   -  -      -  -  10  -  -      -   -
AirBourne_Bds      -  -   -  -      -  -   -  -  -  8.108   -
Ajk2386            -  -   -  -  2.668  -   -  -  -      -   -
AnonymousMouseMan  -  -  10  -      -  -   -  -  -      -   -
Apemangalore       -  -   -  -      -  -   -  -  -      -  10

In [ ]:
# Calculate RMSe and BIAS of Test Data. 
# Baseline predictor for Test data. 
# in Test we will use data from Train to do Calcualation like Total_Raw_Avg.

# This would compalte allthe part of the assigemnt 

# We need to allign it. 


# Ignore Below section, may not be used for final document

In [100]:
df_matrix_item.min().head()

Movie_ID
1    3.0
2    3.0
3    1.0
4    1.0
5    1.0
dtype: float64

In [101]:
df_matrix_item.transpose().min().head()

User_Name
3xHCCH               10.0
AirBourne_Bds        10.0
Ajk2386               6.0
AnonymousMouseMan    10.0
Apemangalore          9.0
dtype: float64

In [102]:
# Try normalizing rating by subtracting row mean 

(df_matrix_item - df_matrix_item.mean(axis=1)).head()
df_matrix_item.mean(axis=1)

User_Name
3xHCCH                   10.0
AirBourne_Bds            10.0
Ajk2386                   6.0
AnonymousMouseMan        10.0
Apemangalore              9.0
AshokePrabha              9.0
BiiivAL                   9.0
Boba_Fett1138             1.0
BrianWatchesMovies       10.0
Bruce654                  8.0
C22Man                    2.0
CANpatbuck3664           10.0
Chrysanthepop            10.0
Cinemadharma              8.0
Crazy_Cracker17          10.0
DICK STEEL               10.0
Darth_Osmosis            10.0
Divergenssi               3.0
Ed-Shullivan              4.0
Ericwenzhe               10.0
Figgy66-915-598470       10.0
GLPman                   10.0
GirishGowda               9.0
Hail_Martini             10.0
Harrymiller1215           7.0
IsabelM11                 1.0
JackieBlu                 1.0
Jared_Andrews             2.0
Joejoesan                 8.0
KillerK1991               8.0
                         ... 
saskCritic               10.0
shawneofthedead          10.0


In [103]:
user_mran_rating = df_matrix_item.transpose().mean().transpose()
user_mran_rating.head()

User_Name
3xHCCH               10.0
AirBourne_Bds        10.0
Ajk2386               6.0
AnonymousMouseMan    10.0
Apemangalore          9.0
dtype: float64

In [104]:
df_matrix_item.loc['Platypuschow']

Movie_ID
1     7.0
2     9.0
3     6.0
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
Name: Platypuschow, dtype: float64

In [105]:
# Comput mean reating by user 
# df_matrix_item['Usr_Mean'] = df_matrix_item.mean(axis= 1)

In [109]:
# df_matrix_item.head()

df_matrix_item_norm = df_matrix_item.subA190015L (df_matrix_item.mean(axis=1), axis=0)

In [113]:
df_matrix_item_norm.head()

Movie_ID           1   2    3   4    5   6    7   8   9    10   11
User_Name                                                         
3xHCCH            NaN NaN  NaN NaN  NaN NaN  0.0 NaN NaN  NaN  NaN
AirBourne_Bds     NaN NaN  NaN NaN  NaN NaN  NaN NaN NaN  0.0  NaN
Ajk2386           NaN NaN  NaN NaN  0.0 NaN  NaN NaN NaN  NaN  NaN
AnonymousMouseMan NaN NaN  0.0 NaN  NaN NaN  NaN NaN NaN  NaN  NaN
Apemangalore      NaN NaN  NaN NaN  NaN NaN  NaN NaN NaN  NaN  0.0

In [119]:
df_matrix_item_norm.loc[['Lady_Targaryen','Platypuschow','TheTopDawgCritic',
'claudio_carvalho',
'neener3707',
'the-movie-guy']]

Movie_ID                1         2         3    4   5   6    7    8    9   \
User_Name                                                                    
Lady_Targaryen         NaN       NaN       NaN  NaN NaN NaN  NaN  1.0 -1.0   
Platypuschow     -0.333333  1.666667 -1.333333  NaN NaN NaN  NaN  NaN  NaN   
TheTopDawgCritic  0.000000       NaN  0.000000  0.0 NaN NaN  NaN  NaN  NaN   
claudio_carvalho  3.000000       NaN  1.000000 -2.0 NaN NaN  2.0  NaN  NaN   
neener3707        2.000000       NaN       NaN -2.0 NaN NaN  NaN  NaN  NaN   
the-movie-guy          NaN       NaN       NaN  NaN NaN NaN  0.5  NaN  NaN   

Movie_ID           10   11  
User_Name                   
Lady_Targaryen    NaN  NaN  
Platypuschow      NaN  NaN  
TheTopDawgCritic  NaN  NaN  
claudio_carvalho -4.0  NaN  
neener3707        NaN  NaN  
the-movie-guy     NaN -0.5

In [ ]:
# To predict the rating we can use option 1 : Get the avg of all th rating for the given movie and set that a
# Avg rating = the rating of movie for the user whi has not yet watched the movie. 


In [133]:
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

flattened = pd.DataFrame(df_matrix_item_norm.to_records())
flattened  = df_matrix_item_norm.reset_index()
flattened.head()

Movie_ID          User_Name   1   2    3   4    5   6    7   8   9   10   11
0                    3xHCCH NaN NaN  NaN NaN  NaN NaN  0.0 NaN NaN  NaN  NaN
1             AirBourne_Bds NaN NaN  NaN NaN  NaN NaN  NaN NaN NaN  0.0  NaN
2                   Ajk2386 NaN NaN  NaN NaN  0.0 NaN  NaN NaN NaN  NaN  NaN
3         AnonymousMouseMan NaN NaN  0.0 NaN  NaN NaN  NaN NaN NaN  NaN  NaN
4              Apemangalore NaN NaN  NaN NaN  NaN NaN  NaN NaN NaN  NaN  0.0

In [126]:
rmse(df_matrix_item_norm,df_matrix_item)

Movie_ID
1     7.688676
2     8.573862
3     7.788287
4     5.837808
5     4.368066
6     6.393747
7     9.267439
8     8.822817
9     8.633752
10    6.636838
11    8.089897
dtype: float64

In [122]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [184]:
# print('User-based CF RMSE: ' + str(rmse(df_matrix_item_norm, df_matrix_item_norm)))